In [1]:
import osmnx as ox
import pandas as pd

# Configure the place and custom filter for highways
place_name = "Seoul, South Korea"
# This custom filter includes major highway types
custom_filter = '["highway"~"motorway|motorway_link|trunk|trunk_link"]'

In [2]:
# Download the street network for highways
G_osm = ox.graph_from_place(place_name, custom_filter=custom_filter)

In [3]:
# Convert to GeoDataFrame
nodes, edges = ox.graph_to_gdfs(G_osm, nodes=True, edges=True)

print(edges.columns)
print(nodes.columns)

Index(['osmid', 'bridge', 'oneway', 'lanes', 'ref', 'name', 'highway',
       'maxspeed', 'width', 'reversed', 'length', 'geometry', 'tunnel',
       'access'],
      dtype='object')
Index(['y', 'x', 'street_count', 'ref', 'highway', 'geometry'], dtype='object')


In [4]:
print(edges.head())

                                                                 osmid bridge  \
u         v          key                                                        
387153848 2139941847 0                                       261477947    yes   
387153860 2139941882 0                                       516417598    yes   
387153879 4593300627 0    [464188578, 464188579, 464188580, 623373663]    yes   
          387153882  0                                       474954574    NaN   
387153882 387153886  0                                       474954574    NaN   

                          oneway lanes          ref   name     highway  \
u         v          key                                                 
387153848 2139941847 0      True     4     23;70;30   강변북로       trunk   
387153860 2139941882 0      True     4           70   강변북로       trunk   
387153879 4593300627 0      True     1          NaN  새남터다리  trunk_link   
          387153882  0      True     4  23;46;70;30   강변북로    

In [5]:
print(edges.columns)

Index(['osmid', 'bridge', 'oneway', 'lanes', 'ref', 'name', 'highway',
       'maxspeed', 'width', 'reversed', 'length', 'geometry', 'tunnel',
       'access'],
      dtype='object')


In [6]:
# Save nodes and edges to CSV
nodes.to_csv("osm_nodes_highways.csv")
edges.to_csv("osm_edges_highways.csv")

In [8]:
edges = pd.read_csv("osm_edges_highways.csv")

# Assuming 'length' is already a column in the edges DataFrame
# If 'length' does not exist, you may need to adjust this part accordingly
new_data = edges[['u', 'v', 'length']].copy()
new_data['travel_capacity'] = 10  # Setting default travel capacity to 0

# Rename the 'length' column to 'travel_time'
new_data.rename(columns={'length': 'travel_time'}, inplace=True)

# Save the new DataFrame to CSV
new_data.to_csv("road_section_data.csv", index=False)

In [9]:
# Load the travel data
travel_data = pd.read_csv("road_section_data.csv")
travel_data.head()

,u,v,travel_time,travel_capacity
0,387153848,2139941847,546.760,10
1,387153860,2139941882,41.005,10
2,387153879,4593300627,554.882,10
3,387153879,387153882,383.087,10
4,387153882,387153886,404.828,10


In [10]:
# Create a DataFrame (assuming your real data is loaded in a similar way)
df = pd.DataFrame(travel_data, columns=['u', 'v', 'travel_time', 'travel_capacity'])
df.head()

,u,v,travel_time,travel_capacity
0,387153848,2139941847,546.760,10
1,387153860,2139941882,41.005,10
2,387153879,4593300627,554.882,10
3,387153879,387153882,383.087,10
4,387153882,387153886,404.828,10


In [11]:
import pandas as pd

# Assuming df is already defined and loaded with the appropriate data
# Initialize the list to store connection data
all_directions_data = []

# Counter to limit the iterations of the outer loop
counter = 0

# Loop: Iterate through each row
for index1, row1 in df.iterrows():
    print(f"Processing iteration {counter+1} for start node: {row1['u']}")
    start_node = row1['u']
    intersection = row1['v']
    
    # Find the subset of rows with matching intersection node
    intersection_subset = df[df['u'] == intersection]

    # Loop through the subset
    for index2, row2 in intersection_subset.iterrows():          
        end_node = row2['v']

        # Print the connection details
        print(f"Considering connection from {start_node} to {end_node} via {intersection}")

        # Create connection data (only if start and end nodes are different)
        if start_node != end_node:
            connection = {
                'intersection': intersection,
                'start_node': start_node,
                'intersection_node': intersection,
                'end_node': end_node,
                'capacity': 10, 
                'delay': 5 
            }
            
            # Add the connection to the list
            all_directions_data.append(connection)

    counter += 1  # Increment the counter after processing each row

# Convert the list to a DataFrame with specified types
all_highway_df = pd.DataFrame(all_directions_data)
all_highway_df = all_highway_df.astype({
    'intersection': 'int64',
    'start_node': 'int64',
    'intersection_node': 'int64',
    'end_node': 'int64',
    'capacity': 'int',
    'delay': 'int'
})

# Save to CSV
all_highway_df.to_csv('intersection_data.csv', index=False)

print("All directions intersection CSV created successfully.")

Processing iteration 1 for start node: 387153848.0
Considering connection from 387153848.0 to 387153860.0 via 2139941847.0
Considering connection from 387153848.0 to 5168879192.0 via 2139941847.0
Processing iteration 2 for start node: 387153860.0
Considering connection from 387153860.0 to 3331239301.0 via 2139941882.0
Considering connection from 387153860.0 to 2139941823.0 via 2139941882.0
Processing iteration 3 for start node: 387153879.0
Considering connection from 387153879.0 to 4593528521.0 via 4593300627.0
Processing iteration 4 for start node: 387153879.0
Considering connection from 387153879.0 to 387153886.0 via 387153882.0
Considering connection from 387153879.0 to 798676076.0 via 387153882.0
Processing iteration 5 for start node: 387153882.0
Considering connection from 387153882.0 to 387153889.0 via 387153886.0
Processing iteration 6 for start node: 387153882.0
Processing iteration 7 for start node: 387153886.0
Considering connection from 387153886.0 to 4243080165.0 via 387153